In [1]:
%matplotlib inline
import sys
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import scipy as sp
import pprint as pp 
# Data reading
import pandas as pd
import csv
import pickle

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# networkx

import networkx as nx
from networkx.algorithms import community
from networkx.algorithms.community import greedy_modularity_communities
from networkx.algorithms.community import k_clique_communities

from community import community_louvain

import scipy.sparse.linalg

In [2]:
# Read networks from file
#G_CDC28=nx.read_weighted_edgelist("./../Data/4932_protein_links_v11_0.txt",comments="#",nodetype=str)
G_CDC28=nx.read_weighted_edgelist("4932.protein.links.v11.0.txt",comments="#",nodetype=str)

print('number of nodes of G:',nx.number_of_nodes(G_CDC28))
print('number of edges of G:',nx.number_of_edges(G_CDC28))

number of nodes of G: 6574
number of edges of G: 922983


In [3]:
# Choose the network to be analyzed below
G0=G_CDC28
# delete those edges with a combined score of <= thershold_score (small confidence)
threshold_score = 700
#threshold_score = 0
for edge in G0.edges: 
    G0.get_edge_data(edge[0],edge[1])
    weight = list(G0.get_edge_data(edge[0],edge[1]).values())
    #print('qwe',weight[0])
    if(weight[0] <= threshold_score):
        G0.remove_edge(edge[0],edge[1])
# restrict to largest connected component
largest_cc = max(nx.connected_components(G0),key=len)
G0=G0.subgraph(largest_cc)

In [4]:
partLouvain = community_louvain.best_partition(G0,resolution=0.5,random_state=1)
number_of_communities = max(list(partLouvain.values()))+1
print('# of partitions for Louvain modularity =',number_of_communities)

# of partitions for Louvain modularity = 29


In [5]:
node_target = '4932.YBR160W' # CDC28
print('The target protein CDC28 belongs to community #',partLouvain[node_target])

The target protein CDC28 belongs to community # 3


In [6]:
# define the communities as separate graphs

#subgraph dict with community number as key and subgraph as value
G_cluster = {}

#node dict with community number as key and node as value
nodes = {}
for i in range(number_of_communities):
    nodes[i] = []

for name, community in partLouvain.items():
    nodes[community].append(name)
    
for key in nodes.keys():
    G_cluster[key] = G0.subgraph(nodes[key])

# local bridges

In [7]:
for community,nodes in G_cluster.items():
    print("community: {}\nLocal bridges".format(community))
    print(sorted(list(nx.local_bridges(nodes)),key=lambda x: x[2],reverse = True))

community: 0
Local bridges
[]
community: 1
Local bridges
[('4932.YLR035C-A', '4932.YGR027W-A', inf), ('4932.YCL024W', '4932.YCL022C', inf), ('4932.YCL024W', '4932.YCL023C', inf), ('4932.YMR068W', '4932.YDR223W', inf), ('4932.YLR030W', '4932.YMR124W', inf), ('4932.YLR030W', '4932.YLR031W', inf), ('4932.YNL234W', '4932.YLR407W', inf), ('4932.YIL029C', '4932.YDL049C', inf), ('4932.YIL029C', '4932.YJR117W', inf), ('4932.YJL182C', '4932.YJL181W', inf), ('4932.YOR300W', '4932.YOR299W', inf), ('4932.YML048W', '4932.YHR094C', inf), ('4932.YCR041W', '4932.YCL066W', inf), ('4932.YKL067W', '4932.YKL066W', inf), ('4932.YHR022C', '4932.YCR027C', inf), ('4932.YGL036W', '4932.YBR057C', inf), ('4932.YCL021W-A', '4932.YKL093W', inf), ('4932.YNL192W', '4932.YLR144C', inf), ('4932.YDL162C', '4932.YDL161W', inf), ('4932.YLR353W', '4932.YBR027C', inf), ('4932.YCR027C', '4932.YJR098C', inf), ('4932.YCR027C', '4932.YGR027W-A', inf), ('4932.YKR013W', '4932.YKR012C', inf), ('4932.YKL130C', '4932.YJL051W', inf)

# Cut Vertices

In [8]:
cut_vertices = []
for community,nodes in G_cluster.items():
    print("community: {}\nCut vertices".format(community))
    print(list(nx.articulation_points(nodes)))
    cut_vertices = cut_vertices+list(nx.articulation_points(nodes))

community: 0
Cut vertices
[]
community: 1
Cut vertices
['4932.YCL066W', '4932.YBR057C', '4932.YGL192W', '4932.YCL055W', '4932.YIR019C', '4932.YER137C', '4932.YDL034W', '4932.YDL035C', '4932.YCL024W', '4932.YER149C', '4932.YLR030W', '4932.YMR124W', '4932.YGR023W', '4932.YJL181W', '4932.YNL212W', '4932.YKR013W', '4932.YMR068W', '4932.YKL093W', '4932.YOL113W', '4932.YOL100W', '4932.YDR423C', '4932.YER132C', '4932.YLR353W', '4932.YKL209C', '4932.YNL279W', '4932.YLR040C', '4932.YLR452C', '4932.YML104C', '4932.YIL029C', '4932.YJR117W', '4932.YNL046W', '4932.YOR264W', '4932.YOR299W', '4932.YMR172W', '4932.YLR154C-H', '4932.YLL053C', '4932.YNL192W', '4932.YLR286C', '4932.YOR386W', '4932.YPL089C', '4932.YGR229C', '4932.YLR429W', '4932.YMR295C', '4932.YDR348C', '4932.YJL023C', '4932.YOR304C-A', '4932.YGR241C', '4932.YIL067C', '4932.YER071C', '4932.YOR140W', '4932.YNL135C', '4932.YPR085C', '4932.YKL033W', '4932.YGR130C', '4932.YNL300W', '4932.YMR182C', '4932.YMR181C', '4932.YBR126W-A', '4932.YKR0

# Impact of Cut Vertices

In [9]:
for community,nodes in G_cluster.items():
    print(community)
    temp = list(nx.articulation_points(nodes))
    for point in temp:
        nodes_copy = nodes.copy()
        nodes_copy.remove_node(point)
        cc = nx.connected_components(nodes_copy)
        cl = sorted([len(graph) for graph in cc],reverse=True) 
        if cl[1] >= 10:
            cl = [point]+cl
            print(cl)

0
1
2
3
4
5
6
7
8
9
['4932.YGL230C', 126, 11, 2, 1, 1]
['4932.YDR124W', 131, 10]
10
11
12
13
14
15
16
17
18
['4932.YDR133C', 86, 15, 1]
['4932.YIL165C', 89, 12, 1]
['4932.YOR343C', 85, 17]
['4932.YCR095C', 91, 11]
['4932.YCR024C-B', 89, 13]
['4932.YLR112W', 88, 14]
['4932.YDR417C', 87, 15]
['4932.YMR304C-A', 89, 13]
['4932.YDL228C', 69, 33]
['4932.YAL004W', 70, 32]
['4932.YBR113W', 72, 30]
['4932.YGR182C', 79, 23]
19
20
21
22
['4932.YLR256W', 185, 11]
23
24
['4932.YHR184W', 56, 14]
['4932.YLR341W', 55, 15]
['4932.YDR523C', 39, 31]
['4932.YDR522C', 38, 32]
['4932.YLR308W', 37, 33]
['4932.YLR307W', 37, 33]
['4932.YAL018C', 52, 18]
['4932.YBL010C', 53, 14, 3]
['4932.YAL067C', 57, 13]
['4932.YLL057C', 58, 11, 1]
25
['4932.YFL067W', 202, 14]
['4932.YHR213W', 201, 10, 5]
26
['4932.YNL092W', 22, 10]
27
['4932.YKL207W', 121, 13]
['4932.YJL192C', 120, 14]
['4932.YPL058C', 119, 15]
['4932.YKR105C', 119, 15]
28
